In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from CKKS_Scheme.ckks_encoder import CKKSEncoder
from CKKS_Scheme.ckks import Params,CKKS
from utils.plaintext import Plaintext
from utils.poly import Polynomial


In [2]:

params = Params(8,  1<<600,1<<1200)
ckks= CKKS(params)
public_key,private_key=ckks.generate_keys()


In [3]:
def read_dataset(url,col):
  df=pd.read_csv(url)
  df.drop("Unnamed: 32", axis=1, inplace=True)
  X = df.iloc[:, col+1:]
  col_name=df.columns[col]
  y = df[col_name]
  for i in range (0,len(df)):
    if y[i]=='M':
      y[i]=1
    else:
      y[i]=0
  X_train, X_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)
  X_train=np.asarray(X_train).astype('float32')
  X_test=np.asarray(X_test).astype('float32')
  y_train=np.asarray(y_train).astype('float32')
  y_test=np.asarray(y_test).astype('float32')
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  print(X_test.shape)
  return X_train, X_test, y_train, y_test


In [4]:
X_train, X_test, y_train, y_test = read_dataset('../dataset/data.csv',1)

(143, 30)


C:\Users\delwy\AppData\Local\Temp\ipykernel_16480\1684536102.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=1
C:\Users\delwy\AppData\Local\Temp\ipykernel_16480\1684536102.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=0


In [5]:
X_train

array([[-0.50754064, -1.5873111 , -0.53291875, ..., -0.29626036,
        -0.09794363, -0.92683077],
       [-0.9010532 , -1.5964096 , -0.92588454, ..., -1.2394603 ,
        -1.1854004 , -1.0006467 ],
       [-1.1103684 ,  0.05040994, -1.1043581 , ..., -1.3858792 ,
        -0.9962085 , -0.9435547 ],
       ...,
       [-0.9317526 , -2.1969075 , -0.9408586 , ..., -1.0250068 ,
        -1.2696624 , -0.3916628 ],
       [-0.73639184, -0.20889577, -0.75833786, ..., -0.72958136,
        -0.23626071, -1.0906105 ],
       [-0.37916064, -0.65472007, -0.43093362, ..., -0.83033943,
        -0.80383676, -1.1350157 ]], dtype=float32)

In [14]:
def custom_relu(x):
    return x*x
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(30,)),
    tf.keras.layers.Dense(2),
    tf.keras.layers.Activation(custom_relu),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=16)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/30
27/27 [==============================] - 1s 2ms/step - loss: 0.9637 - accuracy: 0.5822
Epoch 2/30
27/27 [==============================] - 0s 2ms/step - loss: 0.8643 - accuracy: 0.6009
Epoch 3/30
27/27 [==============================] - 0s 3ms/step - loss: 0.7977 - accuracy: 0.6103
Epoch 4/30
27/27 [==============================] - 0s 2ms/step - loss: 0.7458 - accuracy: 0.6150
Epoch 5/30
27/27 [==============================] - 0s 2ms/step - loss: 0.7072 - accuracy: 0.6408
Epoch 6/30
27/27 [==============================] - 0s 2ms/step - loss: 0.6785 - accuracy: 0.6502
Epoch 7/30
27/27 [==============================] - 0s 2ms/step - loss: 0.6522 - accuracy: 0.6690
Epoch 8/30
27/27 [==============================] - 0s 2ms/step - loss: 0.6309 - accuracy: 0.6901
Epoch 9/30
27/27 [==============================] - 0s 2ms/step - loss: 0.6079 - accuracy: 0.7136
Epoch 10/30
27/27 [==============================] - 0s 2ms/step - loss: 0.5873 - accuracy: 0.7441
Epoch 11/30
27/27 [

In [15]:
def create_model_weights_file(model):
  i=0
  for layer in model.layers:
    if('activation' in layer.name):
      continue
    w = (layer.get_weights()[0]).T
    
    b = (layer.get_weights()[1]).T
    file_name = 'layer'+str(i)+'.npy'
    with open(file_name, 'wb') as f:
        np.save(f, w)
        np.save(f, b)
    i=i+1


In [16]:
create_model_weights_file(model)

In [17]:
with open('layer0.npy', 'rb') as f:
              print(np.load(f))
              print(np.load(f))

[[ 0.05847992  0.10372423 -0.36680582  0.01364348  0.06615735 -0.21566895
  -0.436642   -0.04528108 -0.12435865  0.15005013  0.07096262  0.26047269
   0.11306693 -0.51171887  0.0458674   0.06511447 -0.16024268  0.1782684
  -0.16274862 -0.15963098 -0.18333358 -0.3819804  -0.33792174 -0.48554143
  -0.29534203 -0.15683544  0.05440152 -0.25289032  0.08404428 -0.03637034]
 [-0.10754468 -0.43644127 -0.51116824 -0.46951845 -0.14753978 -0.27654725
   0.30708456  0.10508725  0.36952904  0.4022424  -0.13154568  0.1891353
  -0.00258116  0.07557585  0.13884018 -0.13963626  0.27100983 -0.09432873
  -0.26620883 -0.238154    0.18796888  0.33698624  0.0338765  -0.34890532
  -0.24005976 -0.21684194 -0.20201881 -0.2957581  -0.04373759 -0.24322167]]
[-0.83605677  0.52838075]


In [18]:
def encode_value(a,scale):
    encoder=CKKSEncoder(16,scale)
    a_encoded=encoder.encode(np.array([a]*4))
    a_encoded=Polynomial(a_encoded.coef)
    a_encoded.poly_floor()
    a_pt=Plaintext(a_encoded,scale)
    return a_pt
def encrypt_input(x,scale):
    res=[]
    for item in x:
        item_pt=encode_value(item,scale)
        item_encrypted=public_key.encrypt(item_pt)
        res.append(item_encrypted)
    return res
    
    

In [19]:
class Model:
    def __init__(self, n_features, n_layers, n_layer_neurons, layer_activations, layer_weight_files, encrypted):
        self.n_features = n_features
        self.n_layers = n_layers
        self.n_layer_neurons = n_layer_neurons
        self.layer_activations = layer_activations
        self.weights = []
        self.biases = []
        self.encrypted = encrypted
        layer_number = 0
        b_scale = 1 << 20
        for wt_file in layer_weight_files:
            with open(wt_file, 'rb') as f:
              if self.encrypted == True:
                  w = np.load(f)
                  r, c = np.shape(w)
                  res_w = []
                  for i in range(r):
                      row = []
                      for j in range(c):
                          row.append(encode_value(w[i][j], b_scale))
                      res_w.append(row)
                  b = np.load(f)
                  res_b = []
                #   b_scale=b_scale*(1<<12)
                  for i in b:
                      res_b.append(encode_value(i, b_scale))

                #   if self.layer_activations[layer_number]=='relu':
                #         b_scale=b_scale**2
                  self.weights.append(res_w)
                  self.biases.append(res_b)
                  layer_number += 1

              else:
                self.weights.append(np.load(f).tolist())
                self.biases.append(np.load(f).tolist())

    def __matmul(self, a, b):
        # Input: a to be 2-D (n_layer_neurons[layer_no],n_features)and b to be 1-D list
        # Output : a 1-D list
        output = []
        for i in range(len(a)):
            res = a[i][0]*b[0]
            for j in range(1, len(b)):
                res = a[i][j]*b[j]+res
            output.append(res)
        return output

    def __matadd(self, a, b):
        # Input: a and b to be 1-D list
        # Output : a 1-D list
        # For memory saving b is modified in calculations so be careful about order of inputs

        for i in range(len(a)):
            b[i] = b[i]+a[i]
        return b

    def infer(self, x):
        for i in range(self.n_layers):

            new_x = self.__matmul(self.weights[i], x)
            new_x = self.__matadd(self.biases[i], new_x)

            if (self.layer_activations[i] == 'relu'):

                x = [t**2 for t in new_x]

            else:
                x = new_x
        return x[0]

In [20]:
model_encrypted=Model(30,2,[2,1],['relu','none'],['layer0.npy','layer1.npy'],True)
model_unencrypted=Model(30,2,[2,1],['relu','none'],['layer0.npy','layer1.npy'],False)

In [21]:
normal_acc=0
encrypted_acc=0
limit=len(X_test)
for i in range(limit):
    x=X_test[i].tolist()
    x_encrypted=encrypt_input(x,1<<20)
    normal_res=model_unencrypted.infer(x)
    pt_dash=private_key.decrypt(model_encrypted.infer(x_encrypted))
    pt=np.polynomial.Polynomial(pt_dash.poly.convert_to_list())
    encoder=CKKSEncoder(16,pt_dash.scale)
    pt=encoder.decode(pt)
    encrypted_res=pt[0].real
    #print('Unencryted Inference:'+str(normal_res)+" Encrypted Inference:"+str(encrypted_res))
    if (normal_res>0 and y_test[i]==1) or (normal_res<=0 and y_test[i]==0):
        normal_acc+=1
    if (encrypted_res>0 and y_test[i]==1) or (encrypted_res<=0 and y_test[i]==0):
        encrypted_acc+=1
print('Accuracy on unencrypted Test Data:'+str(normal_acc/limit))
print('Accuracy on encrypted test data:'+str(encrypted_acc/limit))
  

Accuracy on unencrypted Test Data:0.951048951048951
Accuracy on encrypted test data:0.951048951048951
